# Insurance Data EDA

Exploratory Data Analysis for AlphaCare Insurance Solutions using `MachineLearningRating_v3.csv` (converted from pipe-separated .txt, Feb 2014–Aug 2015).

**Guiding Questions**:
- What is the overall Loss Ratio (TotalClaims / TotalPremium)? How does it vary by Province, VehicleType, Gender?
- What are the distributions of TotalPremium, TotalClaims, CustomValueEstimate? Are there outliers?
- Are there temporal trends in claim frequency/severity over the 18-month period?
- Which vehicle make/Model are associated with the highest/lowest claim amounts?

**Notes**:
- `CapitalOutstanding` had comma-separated numbers (e.g., '285700,00'), converted to floats.
- `CrossBorder` and other columns checked for data quality.

In [20]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from src.data_loader import load_data
from src.utils import save_plot

%matplotlib inline
plt.style.use('seaborn-v0_8')

# Load data
df = load_data('../data/MachineLearningRating_v3.csv')

In [21]:
print(df.columns.tolist())


['UnderwrittenCoverID|PolicyID|TransactionMonth|IsVATRegistered|Citizenship|LegalType|Title|Language|Bank|AccountType|MaritalStatus|Gender|Country|Province|PostalCode|MainCrestaZone|SubCrestaZone|ItemType|mmcode|VehicleType|RegistrationYear|make|Model|Cylinders|cubiccapacity|kilowatts|bodytype|NumberOfDoors|VehicleIntroDate|CustomValueEstimate|AlarmImmobiliser|TrackingDevice|CapitalOutstanding|NewVehicle|WrittenOff|Rebuilt|Converted|CrossBorder|NumberOfVehiclesInFleet|SumInsured|TermFrequency|CalculatedPremiumPerTerm|ExcessSelected|CoverCategory|CoverType|CoverGroup|Section|Product|StatutoryClass|StatutoryRiskType|TotalPremium|TotalClaims']


## Data Summarization

In [23]:
# Descriptive statistics
numerical_cols = ['TotalPremium', 'TotalClaims', 'SumInsured', 'kilowatts', 'CustomValueEstimate', 'CapitalOutstanding']
print("Descriptive Statistics:\n", df[numerical_cols].describe())

# Data types
print("\nData Types:\n", df.dtypes)

# Missing values
print("\nMissing Values:\n", df.isnull().sum())

# Check CapitalOutstanding and CrossBorder
print("\nCapitalOutstanding Sample:\n", df['CapitalOutstanding'].head(10))
print("\nCrossBorder Unique Values:\n", df['CrossBorder'].value_counts(dropna=False))

Descriptive Statistics:
        TotalPremium   TotalClaims    SumInsured      kilowatts  \
count  1.000098e+06  1.000098e+06  1.000098e+06  999546.000000   
mean   6.190550e+01  6.486119e+01  6.041727e+05      97.207919   
std    2.302845e+02  2.384075e+03  1.508332e+06      19.393256   
min   -7.825768e+02 -1.200241e+04  1.000000e-02       0.000000   
25%    0.000000e+00  0.000000e+00  5.000000e+03      75.000000   
50%    2.178333e+00  0.000000e+00  7.500000e+03     111.000000   
75%    2.192982e+01  0.000000e+00  2.500000e+05     111.000000   
max    6.528260e+04  3.930921e+05  1.263620e+07     309.000000   

       CustomValueEstimate  
count         2.204560e+05  
mean          2.255311e+05  
std           5.645157e+05  
min           2.000000e+04  
25%           1.350000e+05  
50%           2.200000e+05  
75%           2.800000e+05  
max           2.655000e+07  

Data Types:
 UnderwrittenCoverID          object
PolicyID                     object
TransactionMonth             obje

## Univariate Analysis

In [24]:
# Histogram: CapitalOutstanding
plt.figure(figsize=(10, 6))
sns.histplot(df['CapitalOutstanding'].dropna(), bins=50)
plt.title('Distribution of Capital Outstanding')
save_plot('capital_outstanding_hist.png')

# Histogram: TotalClaims
plt.figure(figsize=(10, 6))
sns.histplot(df['TotalClaims'], bins=50)
plt.title('Distribution of Total Claims')
save_plot('total_claims_hist.png')

# Bar chart: Province
plt.figure(figsize=(10, 6))
df['Province'].value_counts().plot(kind='bar')
plt.title('Distribution of Policies by Province')
plt.xticks(rotation=45)
save_plot('province_bar.png')

## Bivariate/Multivariate Analysis

In [25]:
# Scatter plot: Premium vs Claims by PostalCode
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='TotalPremium', y='TotalClaims', hue='PostalCode', size=10, legend=False)
plt.title('Premium vs Claims by PostalCode')
save_plot('premium_vs_claims_scatter.png')

# Correlation matrix
corr_matrix = df[numerical_cols].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
save_plot('correlation_matrix.png')

## Loss Ratio Analysis

Calculate `LossRatio = TotalClaims / TotalPremium` and analyze by Province, VehicleType, Gender.

In [26]:
# Avoid division by zero
df['LossRatio'] = df['TotalClaims'] / df['TotalPremium'].replace(0, float('nan'))
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='Province', y='LossRatio', estimator='mean')
plt.title('Average Loss Ratio by Province')
plt.xticks(rotation=45)
save_plot('loss_ratio_province.png')

## Temporal Trends

Analyze claim frequency over TransactionMonth.

In [29]:
df['HasClaim'] = df['TotalClaims'] > 0
claim_freq = df.groupby(df['TransactionMonth'].dt.to_period('M'))['HasClaim'].mean()
plt.figure(figsize=(12, 6))
claim_freq.plot(kind='line')
plt.title('Claim Frequency Over Time')
plt.xlabel('Transaction Month')
plt.ylabel('Claim Frequency')
save_plot('claim_freq_time.png')

## Outlier Detection

In [30]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, y='TotalClaims', x='VehicleType')
plt.title('Box Plot of Total Claims by Vehicle Type')
plt.xticks(rotation=45)
save_plot('total_claims_boxplot.png')

## Vehicle Make/Model Analysis

Identify make/Model with highest/lowest claim amounts.

In [32]:
claims_by_make = df.groupby('make')['TotalClaims'].mean().sort_values()
plt.figure(figsize=(12, 6))
pd.concat([claims_by_make.head(5), claims_by_make.tail(5)]).plot(kind='bar')
plt.title('Top/Bottom 5 Makes by Average Claim Amount')
plt.xticks(rotation=45)
save_plot('claims_by_make.png')